In [28]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import matplotlib.pyplot as plt
import codecs
%matplotlib inline  

In [68]:
def review_parse(review):
    review_id=review['id']
    reviewer_name=review.find('a',class_='user').get_text()
    review_date=review.find('a',class_='reviewDate').get_text()
    score_dict={"it was amazing":5,"really liked it":4,"liked it":3,"it was ok":2,"did not like it":1}
    reviewer_rating=review.find('span',class_='staticStar',text=True).get_text()
    reviewer_score=score_dict[reviewer_rating]
    review_boxes=review.findAll('span', class_='readable')[0].findAll('span')
    review_text=[review_box.get_text() for review_box in review_boxes[min(1,len(review_boxes)-1):]]
    
    return ([review_id],[reviewer_name],[review_date],[reviewer_rating],[reviewer_score],[review_text])
    

In [79]:
p = requests.get("https://www.goodreads.com/book/show/9361589-the-night-circus"+"?sort=newest&text_only=true")
soupit = BeautifulSoup(p.text,"lxml")
test1=soupit.find('div',class_='review')

In [80]:
review_parse(test1)

(['review_1963503865'],
 ['Marysia Anna'],
 ['May 12, 2017'],
 ['really liked it'],
 [4],
 [["I had high expectations for this book, but I was slightly disappointed (note: slightly). I finished most of this (312 pages!) this afternoon. I felt a 'Caraval' vibe at first, but then...Ceila and Marco happened. My thoughts:1. I thought they were supposed to be opponents? 2. Where did it come from?3. Why does it feel so rushed?4. What about Isobel?I would've rated this 5 stars, but I wasn't a fan of their relationship. Though, the rest was great. The circus concept was really mysterious and cool. The other characters (Bailey, Poppet, and Widget) were great as well. "]])

In [81]:
def reviews_find(soupy,book_address):
     
    review_ids=[]
    reviewer_names=[]
    review_dates=[]
    reviewer_ratings=[]
    reviewer_scores=[]
    review_texts=[]    
    
    book_address=book_address+"?sort=newest&text_only=true"
    while book_address:

        response = requests.get(book_address)

        if response.status_code == 200:
            #print("")
            #print("ok",book_address)
            soup = BeautifulSoup(codecs.getdecoder("unicode_escape")(response.text)[0],"lxml")    
            
            for review in soup.findAll('div', class_="review"):
                if "rated it" in review.get_text(): 
                    result=review_parse(review)
                    review_ids=review_ids+result[0]
                    reviewer_names=reviewer_names+result[1]
                    review_dates=review_dates+result[2]
                    reviewer_ratings=reviewer_ratings+result[3]
                    reviewer_scores=reviewer_scores+result[4]
                    review_texts=review_texts+result[5] 
          

            try:
                if book_address == "http://www.goodreads.com"+ soup.findAll('a', 
                                class_="next_page")[0]['onclick'].split("'")[1]:
                    book_address = ""
                else:
                    book_address = "http://www.goodreads.com"+ soup.findAll('a', 
                                class_="next_page")[0]['onclick'].split("'")[1]
                
            except:
                book_address = ""
        else:
            print("reviews not found")
            
    return [review_ids],[reviewer_names],[review_dates], [reviewer_ratings],[reviewer_scores],[review_texts]
    

In [82]:
def isbn_find (soupy):
    if soupy.findAll('div',class_="infoBoxRowTitle")[0].get_text()=='ISBN':
        isbn = soupy.findAll('div',class_="infoBoxRowItem")[0].get_text().split("\n")[1].strip()
    elif soupy.findAll('div',class_="infoBoxRowTitle")[1].get_text()=='ISBN':
         isbn = soupy.findAll('div',class_="infoBoxRowItem")[1].get_text().split("\n")[1].strip()
    else: isbn = 'NA'
    return isbn

In [88]:
def parse_page (link):
    """
    website_response in unicode text -> dictionary of attributes
    
    """
    webpage = requests.get(link)
    book = {}
    
    soup = BeautifulSoup(webpage.text, "lxml")
    
    try:
        book["title"] = [soup.find('h1',class_='bookTitle').get_text().strip()]
    except:
        book["title"] =[""]
    
    try:
        book["authors"] = [[author.get_text().strip() 
                       for author in soup.findAll('a',class_="authorName")]]
    except:
        book["authors"] = [""]
    
    try:
        book["rating"] = [soup.find('span',class_="value rating").get_text().strip()]
    except:
        book["rating"] = [""]
    
    try:
        book["isbn"] = [isbn_find(soup)]
    except:
        book["isbn"] = [""]
    
    try:
        book["date"] = [soup.findAll('div',class_="row")[1].get_text().split("\n")[2].strip()]
    except:
        book["date"] = [""]
    
    try:
        book["description"] = [soup.find('div',class_="readable stacked").findAll('span')[-1].get_text()]
    except:
        book["description"] = [""]
    
    try:
        book["awards"] = [[award.get_text() for award in soup.findAll('a',class_="award")]]
    except:
        book["awards"] = [""]
    
    #result=reviews_find(soup,link)
    #try:
    result=reviews_find(soup,link)
    book["review_ids"]=result[0]
    book["reviewer_names"]=result[1]
    book["review_dates"]=result[2]
    book["reviewer_ratings"]=result[3]
    book["reviewer_scores"]=result[4]
    book["review_texts"]=result[5] 
    #except:
        #book["review_ids"]=[""]
        #book["reviewer_names"]=[""]
        #book["review_dates"]=[""]
        #book["reviewer_ratings"]=[""]
       # book["reviewer_scores"]=[""]
        #book["review_texts"]=[""]
    
       
        #print("fail to get reivew")
        
    book["link"] = link
    book["page"] = webpage.text
    
    return book

In [89]:
df = pd.DataFrame()
list_address="http://www.goodreads.com/genres/new_releases/romance"

while list_address:

    list_response = requests.get(list_address)

    if list_response.status_code == 200:
        print("")
        print("ok",list_address)
        soup = BeautifulSoup(list_response.text,"lxml")
        book_links = ["http://www.goodreads.com" + book.find("a")["href"] for book in soup.find_all("div",class_="coverWrapper")]
        
        
        try:
            list_address = "http://www.goodreads.com"+ soup.find('a',class_='next_page')['href']
        except:
            list_address = ""
    else:
        print("website not found")

    for num, link in enumerate(book_links):
     
        book_results = parse_page(link)
        print(link)
        
        df=df.append(pd.DataFrame(book_results),ignore_index=True)



ok http://www.goodreads.com/genres/new_releases/romance
http://www.goodreads.com/book/show/30653853-the-upside-of-unrequited
http://www.goodreads.com/book/show/30312700-alex-approximately


KeyboardInterrupt: 

In [90]:
df

,authors,awards,date,description,isbn,link,page,rating,review_dates,review_ids,review_texts,reviewer_names,reviewer_ratings,reviewer_scores,title
0,[Becky Albertalli],[],April 11th 2017,Seventeen-year-old Molly Peskin-Suso knows all...,0062348701,http://www.goodreads.com/book/show/30653853-th...,"<!DOCTYPE html>\n<html class=""desktop"">\n\n\n<...",4.17,"[May 12, 2017, May 12, 2017, May 12, 2017, May...","[review_1998467435, review_1997758994, review_...",[[3.5 Stars. Just like Simon Vs. The Homo Sapi...,"[Sophia Jones, Kaitie, Ashley Peet, Azee, Tiff...","[liked it, really liked it, it was amazing, it...","[3, 4, 5, 5, 5, 2, 4, 4, 3, 4, 5, 4, 4, 5, 5, ...",The Upside of Unrequited
1,[Jenn Bennett],[],April 4th 2017,The one guy Bailey Rydell can’t stand is actua...,148147877X,http://www.goodreads.com/book/show/30312700-al...,"<!DOCTYPE html>\n<html class=""desktop"">\n\n\n<...",4.07,"[May 12, 2017, May 12, 2017, May 12, 2017, May...","[review_1998055552, review_1907935087, review_...","[[3.5 Stars!], [3.75 STARS], [[ one thing the ...","[Pehel , Laura, Erin, Love, Beck, Melissa, Mar...","[liked it, really liked it, liked it, it was a...","[3, 4, 3, 5, 5, 4, 5, 4, 3, 3, 5, 4, 4, 4, 4, ...","Alex, Approximately"


In [ ]:
df.to_pickle("romance_novels.pkl")

In [ ]:
df.to_csv("romance_novels.csv")

In [ ]:
import nltk
nltk.download()

In [9]:
len(df)

100

In [91]:
pwd

'/Users/brechann/workspace/goodreads'